In [2]:
%%file polybas.m
function T = polybas(kmin,kmax,Np,kgrid)

    % Np: 多項式の次数-1
    % Ng: グリッドの数
    Ng = size(kgrid,1);
    x = (2/(kmax-kmin))*(kgrid-kmin) - 1;
    
    % 基底関数の行列(NgxNp)を再帰的に求める
    T = zeros(Ng,Np);
    T0 = ones(Ng,1);
    T1 = x;
    T2 = 2*x.*T1 - T0;
    T(:,1) = T1;
    T(:,2) = T2;
    
    for i=3:Np-1
        T(:,i) = 2*x.*T(:,i-1) - T(:,i-2);
    end
    
    T = [T0 T(:,1:(Np-1))];

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\polybas.m'.


In [3]:
%%file polygrid.m
function k = polygrid(kmin,kmax,N)

    temp = linspace(0,N-1,N)'; %[0:N-1:1]'; % チェビシェフ極値点
    x = -cos((pi/(N-1))*temp);
%    temp = linspace(0,N-1,N)'; %[0:N-1:1]'; % チェビシェフゼロ点
%    x = [0; -cos((pi/2/(N-1))*(2*temp - 1))]
    
    % xからkに変換
    k = 0.5*(kmax-kmin)*(x+1) + kmin;

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\polygrid.m'.


In [4]:
%%file EulerEq_cheb.m
function  res = EulerEq_cheb(cons,m,capital,theta)
% cを与えたときのオイラー方程式の残差を返す関数

wealth = capital.^m.alpha + (1.-m.delta).*capital;

kprime = wealth - cons;
% トリック: k'は正の値しか取らない
kprime = max(m.kgrid(1),kprime);

% 次期の政策関数を線形補間: m.nk=21のときは政策関数の形がおかしい???
%cnext = interp1(m.kgrid,cfcn,kprime,'linear','extrap');
% 次期の価値関数をスプライン補間
%cnext = interp1(m.kgrid,cfcn,kprime,'spline');
% 次期の価値関数を多項式補間
T = polybas(m.kmin,m.kmax,m.nk,kprime);
cnext = T*theta;

% オイラー方程式
res = (1/cons) - m.beta*(1/cnext)*(m.alpha*kprime.^(m.alpha-1) + (1.-m.delta));
 
return

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\EulerEq_cheb.m'.


In [5]:
%%file CRRA.m
function util = CRRA(cons,gamma)
% Function CRRA
%  utility = CRRA( consumption, gamma )
%
% Purpose:
%  Compute CRRA utility function
%
%  Record of revisions:
%     Date     Programmer  Description of change
%  ==========  ==========  =====================
%  10/05/2002  T. Yamada   Original code

if gamma ~= 1;
    util = cons.^(1-gamma)./(1-gamma);
else
    util = log(cons);
end

return

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\CRRA.m'.


In [6]:
%%file mu_CRRA.m
function mu = mu_CRRA( cons, gamma )
% Function mu_CRRA
%  marginal_utility = mu_CRRA( consumption, gamma )
%
% Purpose:
%  Compute marginal utility of CRRA-type function
%
%  Record of revisions:
%     Date     Programmer  Description of change
%  ==========  ==========  =====================
%  02/22/2016  T. Yamada   Original code

mu = cons.^-gamma;

return

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\mu_CRRA.m'.


In [7]:
%%file nti_cheb.m
function [cfcn0 dif] = nti_cheb(m)

options = optimoptions('fsolve','Display','none'); % fsolveのオプション(最適化の結果を非表示にする)

% *** 収束の基準 ***
it = 1;          % ループ・カウンター
dif2 = 1.0;      % 政策関数の繰り返し誤差
options.TolFun = 1.0e-10; % fsolveのオプション(最適化の許容誤差)

% disp(' ')
% disp('-+- Solve a neoclassical growth model with time iteration -+-');
% disp(' ')

%% STEP 1(b): 政策関数の初期値を当て推量
% 解析解 (for k'=g(k))
p_true = m.beta*m.alpha*(m.kgrid.^m.alpha);

% 政策関数の初期化
cfcn0 = m.kgrid;
%cfcn0 = m.css/m.kss*m.kgrid;
%cfcn0 = m.kgrid.^m.alpha - p_true;
%cfcn0 = m.css*ones(nk,1);
cfcn1 = zeros(m.nk,1);

% 繰り返し誤差を保存する変数を設定 
dif = zeros(2,m.maxiter);

%% STEP 4: 政策関数を繰り返し計算
while (it < m.maxiter && dif2 > m.tol)

%     fprintf('iteration index: %i \n', it);
%     fprintf('policy function iteration error: %e\n', dif2);

    theta = m.invT*cfcn0;

    for i = 1:m.nk

        capital = m.kgrid(i);
        wealth = capital.^m.alpha + (1.-m.delta).*capital;

        % MATLABの最適化関数(fsolve)を使って各グリッド上の政策関数の値を探す
        % 最適化の初期値は古い政策関数の値
        cons = fsolve(@EulerEq_cheb,cfcn0(i,1),options,m,capital,theta);
        % 最適化の初期値は定常状態の値: これでは解けない
        % cons = fsolve(@EulerEq2,css,options,m,capital,cfcn0);
        cfcn1(i,1) = cons;
        kprime = wealth-cons;
        % グリッドごとに最適化の結果を確認
        %disp([cons capital wealth kprime]);
        %pause

    end

    % 繰り返し計算誤差を確認
    dif2 = max(abs(cfcn1-cfcn0));
    
    % 収束途中の繰り返し計算誤差を保存
    dif(2,it) = dif2;

    % 政策関数をアップデート
    cfcn0 = cfcn1;

    it = it + 1;

end

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\nti_cheb.m'.


In [8]:
%%file calcerr.m
function err = calcerr(m,cfcn0)
%% オイラー方程式から誤差を測定
% 元のグリッドではオイラー方程式の誤差はゼロになるため、グリッドを細かくとる
theta = m.invT*cfcn0;
kgrid_err = linspace(m.kmin,m.kmax,(m.nk-1)*10+1)';
T = polybas(m.kmin,m.kmax,m.nk,kgrid_err);
cons = T*theta;
%cons = interp1(m.kgrid,cfcn0(:,1),kgrid_err);
LHS  = mu_CRRA(cons, m.gamma);

kp   = kgrid_err.^m.alpha + (1-m.delta)*kgrid_err - cons;
T = polybas(m.kmin,m.kmax,m.nk,kp);
cnext = T*theta;
%cnext = interp1(m.kgrid, cfcn0(:,1), kp);
rent = m.alpha.*kp.^(m.alpha-1.0) - m.delta;
RHS  = m.beta.*(1.+rent).*mu_CRRA(cnext,m.gamma);

err  = RHS./LHS-1.0;

Created file 'C:\Users\takeki\Documents\keizai-seminar\chapter4\Matlab\3_TI_Optimization_Chebyshev\calcerr.m'.


In [9]:
% カリブレーション
m.beta  = 0.96; % 割引因子
m.gamma = 1.0;  % 相対的危険回避度(異時点間の代替の弾力性の逆数)
m.alpha = 0.40; % 資本分配率
m.delta = 1.00; % 固定資本減耗(delta=1.0のときは解析解が存在)

% 定常状態の値
m.ykss = (1/m.beta-1+m.delta)/m.alpha;
m.kss = m.ykss^(1/(m.alpha-1));
m.yss = m.ykss*m.kss;
m.css = m.yss-m.delta*m.kss;

% m.kmax = 0.5;   % 資本グリッドの最大値
% m.kmin = 0.05;  % 資本グリッドの最小値 (0にすると生産が出来なくなる)
m.kmax = 1.2*m.kss;  % 資本グリッドの最大値
m.kmin = 0.8*m.kss;  % 資本グリッドの最小値 (0にすると生産が出来なくなる)

m.maxiter = 1000; % 繰り返し計算の最大値
m.tol  = 1.0e-8;  % 許容誤差(STEP 2)

In [15]:
norms = zeros(3,2);
times = zeros(3,2);

nkvec = [3 5 9]';

for i=1:3

    %% STEP 1(a): グリッド生成
    m.nk = nkvec(i);
    m.kgrid = polygrid(m.kmin,m.kmax,m.nk);
    m.T = polybas(m.kmin,m.kmax,m.nk,m.kgrid);
    m.invT = inv(m.T);

    % time iteration
    tic;
    [cfcn0 dif] = nti_cheb(m);
    times(i,1) = toc;

    err = calcerr(m,cfcn0);
    norms(i,:) = log10([mean(abs(err)) max(abs(err))]);
    
end

times(:,2) = times(:,1)/times(1,1);

disp(" Euler equation errors");
disp([round(norms,2)]);
disp(" Elasped time");
disp([round(times,2)]);

 Euler equation errors
   -3.5000   -3.2300
   -5.8000   -5.4900
   -7.6800   -7.6800

 Elasped time
    0.0800    1.0000
    0.1300    1.5400
    0.2000    2.3500




In [11]:
%% 最終的な政策関数が得られてから貯蓄関数を計算
% pfcn0 = m.kgrid.^m.alpha + (1-m.delta)*m.kgrid - cfcn0;

In [12]:
%% 解析的解
% p_true = m.beta*m.alpha*(m.kgrid.^m.alpha);

In [13]:
% figure;
% plot(m.kgrid, pfcn0, '-', 'color', 'blue', 'linewidth', 3); hold('on');
% plot(m.kgrid, p_true, '--', 'color', 'red', 'linewidth', 3);
% plot(m.kgrid, m.kgrid, ':', 'color', 'black', 'linewidth', 2); hold('off');
% xlabel('今期の資本保有量：k', 'Fontsize', 16);
% ylabel("次期の資本保有量：k'", 'Fontsize', 16);
% xlim([0, m.kmax]);
% legend('近似解', '解析的解', '45度線', 'Location', 'NorthWest');
% grid on;
% set(gca,'Fontsize', 16);

In [14]:
% %err2 = csvread("err_ndp.csv");
% %figure;
% plot(kgrid_err, abs(err), '-', 'color', 'blue', 'linewidth', 3); hold('on');
% %plot(kgrid_err, abs(err2), '--', 'color', 'red', 'linewidth', 3); hold('off');
% xlabel('資本保有量：k', 'Fontsize', 16);
% ylabel('オイラー方程式誤差(絶対値)', 'Fontsize', 16);
% %legend('TI', 'VFI', 'Location', 'NorthEast');
% grid on;
% set(gca,'Fontsize', 16);